# Reverse deskewing

In [5]:
from skimage.io import imread
import napari
import pyclesperanto_prototype as cle
import numpy as np



#Read lattice file
rbc = imread("../sample_data/RBC_lattice.tif")

voxel_size_x_in_microns = 0.1449922
voxel_size_y_in_microns = 0.1449922
voxel_size_z_in_microns = 0.3

deskewing_angle_in_degrees = 30


viewer = napari.Viewer()
viewer.add_image(rbc)


#Perform deskewing on rbc image  
deskewed = cle.deskew_y(rbc, 
                        angle_in_degrees=deskewing_angle_in_degrees, 
                        voxel_size_x=voxel_size_x_in_microns, 
                        voxel_size_y=voxel_size_y_in_microns, 
                        voxel_size_z=voxel_size_z_in_microns)

viewer.add_image(deskewed)

print("Shape of original image", rbc.shape )

print("Shape of deskewed image",deskewed.shape)


If you want to compute separable approximations, please install it with
pip install scikit-tensor-py3
Shape of original image (150, 118, 209)
Shape of deskewed image (59, 416, 209)


### Reverse the deskewing transformation from deskewed volume back into the original volume

Main differences with deskewing are 
* Transforms are concatenated in the reverse order, compared to `deskew_y` function
* Shear matrix: shearing angle is negative
* Rotation matrices: negative of shearing angle and -90 for rotation
* Scaling matrix is inverted using np.linalg.inv

In [93]:
import math 

scaling_factor = 1

#Define tranform
transform = cle.AffineTransform3D()

#Shear Transformation
shear_mat = cle.AffineTransform3D()
shear_mat.shear_in_x_plane(angle_y_in_degrees = -(90-deskewing_angle_in_degrees))
transform._concatenate(shear_mat) #concatenate instead of pre_concatenate

#Rotate along deskewing angle
rotate_transform = cle.AffineTransform3D()
rotate_transform.rotate(angle_in_degrees=-(90-deskewing_angle_in_degrees), axis=0)
transform._pre_concatenate(rotate_transform) #pre_concatenate instead of concatenate

#Invert scaling
new_dz=math.sin(deskewing_angle_in_degrees * math.pi/180.0)*voxel_size_z_in_microns
scale_factor_z=(new_dz/voxel_size_y_in_microns)*scaling_factor
    
scale_transform = cle.AffineTransform3D()
scale_transform.scale(scale_x = scaling_factor, scale_y = scaling_factor, scale_z=scale_factor_z)
scale_inverse = np.linalg.inv(scale_transform._matrix)
transform._pre_concatenate(scale_inverse)  #pre_concatenate instead of concatenate

#Rotate -90 degrees
rotate_transform1 = cle.AffineTransform3D()
rotate_transform1.rotate(angle_in_degrees=-(90), axis=0)
transform._pre_concatenate(rotate_transform1)#pre_concatenate instead of concatenate

#Perform above transformation first, which returns a deskewed volume
transformed = cle.affine_transform(source=deskewed, transform = transform, auto_size=True)
viewer.add_image(transformed)


#Translate image so it stays within bounds

from itertools import product
nz, ny, nx = deskewed.shape
original_bounding_box = [list(x) + [1] for x in product((0, nx), (0, ny), (0, nz))]
# transform the corners using the given affine transform 
transformed_bounding_box = np.asarray(list(map(lambda x: transform._matrix@x, original_bounding_box)))
translation = transformed_bounding_box[1][2]

#Calculate translation based on bounding box corner
transform_translate =  cle.AffineTransform3D()
transform_translate.translate (translate_z = translation)


#subtract opposite vertices for new shape
new_shape = np.abs(np.rint(transformed_bounding_box[6] - transformed_bounding_box[1])).astype(int)[0:3].tolist()[::-1]

#Create new shape and perform affine transformation using this as destination
transformed_translated = cle.create(new_shape)

#pass volume with new shape as destination and apply translation
cle.affine_transform(source=transformed, destination = transformed_translated, transform = transform_translate)

viewer.add_image(transformed_translated)


<Image layer 'transformed_translated [1]' at 0x1824f267430>

In [54]:
print("Checking if the shape of reverse transformed object is same as the original volume")
transformed_translated.shape == rbc.shape


Checking if the shape of reverse transformed object is same as the original volume


True

In [42]:
#print so np arrays have only 4 decimal places; makes it cleaner
np.set_printoptions(precision=4)

transformed_bounding_box

array([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00],
       [ 0.0000e+00,  1.1800e+02, -5.1095e+01,  1.0000e+00],
       [ 0.0000e+00,  1.7291e-13, -2.0106e+02,  1.0000e+00],
       [ 0.0000e+00,  1.1800e+02, -2.5215e+02,  1.0000e+00],
       [ 2.0900e+02,  0.0000e+00,  0.0000e+00,  1.0000e+00],
       [ 2.0900e+02,  1.1800e+02, -5.1095e+01,  1.0000e+00],
       [ 2.0900e+02,  1.7291e-13, -2.0106e+02,  1.0000e+00],
       [ 2.0900e+02,  1.1800e+02, -2.5215e+02,  1.0000e+00]])

## Reverse transformation of ROI
Add shapes layer and draw ROI on the deskewed stack.
Run the cell after this to get bounding box in 3D

`Z axes being returned after transformation are not accurate.. Need to fix that`

In [67]:
#Code to read shapes layer and get bounding box in 3D

import numpy as np 
shapes_layer = viewer.layers["Shapes"]
shape_types = shapes_layer.shape_type
shapes = shapes_layer.data


shape = shapes[0]
start = np.rint(np.min(shape, axis=0))
stop = np.rint(np.max(shape, axis=0))

_,y0,x0 = np.stack([start,stop])[0].astype(int)
_,y1,x1 = np.stack([start,stop])[1].astype(int)

crop_shape = (stop - start).astype(int).tolist()
z0 = 0
z1 = deskewed.shape[0]-1

from itertools import product

crop_bounding_box = [list(x)+[1] for x in product((x0,x1),(y0,y1),(z0,z1))] 
#crop_bounding_box = [list(x)+[1] for x in product((z0,z1),(y0,y1),(x0,x1))] 
crop_bounding_box

[[47, 127, 0, 1],
 [47, 127, 58, 1],
 [47, 246, 0, 1],
 [47, 246, 58, 1],
 [179, 127, 0, 1],
 [179, 127, 58, 1],
 [179, 246, 0, 1],
 [179, 246, 58, 1]]

In [99]:
#reverse_transform = cle.AffineTransform3D()
#transform_translate._pre_concatenate(transform)
#reverse_transform._concatenate(transform_translate)

#Working on using suitable transformations

crop_transformed_bounding_box=np.asarray(list(map(lambda x:transform._matrix@x,crop_bounding_box)))
crop_transformed_bounding_box1=np.asarray(list(map(lambda x:transform_translate._matrix@x,crop_transformed_bounding_box)))
crop_transformed_bounding_box1

array([[-4.7000e+01, -5.2788e-14,  1.1248e+02, -1.0000e+00],
       [-4.7000e+01, -1.1600e+02,  1.6271e+02, -1.0000e+00],
       [-4.7000e+01, -1.0225e-13,  1.6999e+02, -1.0000e+00],
       [-4.7000e+01, -1.1600e+02,  2.2022e+02, -1.0000e+00],
       [-1.7900e+02, -5.2788e-14,  1.1248e+02, -1.0000e+00],
       [-1.7900e+02, -1.1600e+02,  1.6271e+02, -1.0000e+00],
       [-1.7900e+02, -1.0225e-13,  1.6999e+02, -1.0000e+00],
       [-1.7900e+02, -1.1600e+02,  2.2022e+02, -1.0000e+00]])

In [97]:
slice0 = np.rint(np.min(crop_transformed_bounding_box1,axis=0))
slice1= np.rint(np.max(crop_transformed_bounding_box1,axis=0))
print(slice0)
print(slice1)

[  47.    0. -169. 3137.]
[ 179.  116.  -61. 8642.]


##### Test function for returning only the transform for deskewing.. 

In [ ]:
import pyclesperanto_prototype as cle
from pyclesperanto_prototype import Image, affine_transform, AffineTransform3D
import numpy as np
import math
from itertools import product

#GEt transformation for deskewing
def deskew_y_transform(input_image ,
             angle_in_degrees: float = 30,
             voxel_size_x: float = 1,
             voxel_size_y: float = 1,
             voxel_size_z: float = 1,
             scaling_factor: float = 1
             ):
    #Return only the affine transform

    # shear in the X plane towards Y
    transform = AffineTransform3D()
    transform.shear_in_x_plane(angle_y_in_degrees = 90 - angle_in_degrees)

    # rotate the stack to get proper Z-planes; rotate 90 - angle around X-axis
    transform.rotate(angle_in_degrees = 90-angle_in_degrees, axis=0)

    # make voxels isotropic, calculate the new scaling factor for Z after shearing
    #https://github.com/tlambert03/napari-ndtiffs/blob/092acbd92bfdbf3ecb1eb9c7fc146411ad9e6aae/napari_ndtiffs/affine.py#L57
    new_dz=math.sin(angle_in_degrees * math.pi/180.0)*voxel_size_z
    scale_factor_z=(new_dz/voxel_size_y)*scaling_factor
    transform.scale(scale_x = scaling_factor, scale_y = scaling_factor, scale_z=scale_factor_z)

    # correct orientation so that the new Z-plane goes proximal-distal from the objective.
    transform.rotate(angle_in_degrees=90, axis=0)

    #Calculate new affine transform 
    new_shape, new_affine_transform, translation = cle._tier8._affine_transform._determine_translation_and_bounding_box(input_image, transform)

    #Return the affine transform and the translation
    return new_affine_transform, translation


In [91]:
viewer.add_image(rbc[0:63,0:118,102:142])

<Image layer 'Image [1]' at 0x20f955fd340>

In [17]:
crop_shape = (stop - start).astype(int).tolist()
crop_shape

[49, 45]